In [17]:
import sys
sys.path.append("./work")

from pyspark.sql import SparkSession

In [27]:
spark = SparkSession.builder \
                    .appName("Spark Streaming") \
                    .master('spark://spark-master:7077') \
                    .config("spark.executor.memory", "1g") \
                    .config("spark.sql.shuffle.partitions", "100") \
                    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
                    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                    .config('spark.sql.warehouse.dir', f's3a://lakehouse/') \
                    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0') \
                    .enableHiveSupport() \
                    .getOrCreate()

In [21]:
spark

In [30]:
delta_df = spark.read.format("delta").load("s3a://lakehouse/streaming/test/taxi_trends")
delta_df.where(delta_df.pickup_location==113).show()

+---------------+------------------+----------------+
|pickup_location| avg_trip_distance|avg_total_amount|
+---------------+------------------+----------------+
|            113|3.5520000000000005|          20.296|
+---------------+------------------+----------------+



In [31]:
spark.stop()